In [5]:
! pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import json
import pandas as pd
import urllib.request
import environment as env
import openai
from datetime import datetime


CLIENT_ID = env.CLIENT_ID
CLIENT_SECRET = env.CLIENT_SECRET
NAVER_NEWS_QUERY_URL = env.NAVER_NEWS_QUERY_URL
OPENAI_API_KEY= ''
# print(CLIENT_ID, CLIENT_SECRET, NAVER_NEWS_QUERY_URL, OPENAI_API_KEY)

MODEL = "gpt-3.5-turbo"

In [8]:
# 네이버 뉴스 크롤링
# 검색어에 따라 최신 뉴스 결과 출력

def get_search_news(keyword: str, dp_cnt=20, sort='date') -> list:
  '''
    검색어를 넣어서 네이버 뉴스를 수집해서 가져온다
    검색할때 20개 가져오고, 최신순으로 가져온다 => 단 변경가능하다(외부조정가능)
      Args
        keyword `str` : 검색어
        ...
      Returns
        ...
  '''
  # 키워드 인자값을 인코딩 변환에 삽입

  encText = urllib.parse.quote(keyword)
  # 파라미터 구성 : 키=값&키=값&키=값 <- http에서 GET 방식으로 데이터 전송시 포멧
  PARAM = f'display={dp_cnt}&sort={sort}&query={encText}'  # 구조가 보인다
  url = f"https://{NAVER_NEWS_QUERY_URL}?{PARAM}"  # https 프로토콜 구조

  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id", CLIENT_ID)
  request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode == 200):
    tmp = json.load(response)
    #print( tmp['items'] )
    return tmp.get('items')
  else:
    # 단 통신 오류등의 문제라면 로그 처리
    return []


tmp = get_search_news('부동산', dp_cnt=5)
print(tmp)

# 데이터 클리닝 작업 
pattern = [('<b>', ''), ('</b>', ''), ('&quot;', '"'),('&apos;', "'"), ('&amp;', '&')]

def clean_str_ex(ori_txt, pattern):
  while pattern:
    old_str, new_str = pattern.pop()  # 뒤에서 하나씩 제거하여 리턴
    #print( old_str, new_str )
    ori_txt = ori_txt.replace(old_str, new_str)
  return ori_txt

newDatas = [{'tit': clean_str_ex(n.get('title'),  pattern.copy()),
             'desc': clean_str_ex(n['description'],  pattern[:]),
             'date': clean_str_ex(n.get('pubDate'),  pattern[:]),
             } for n in tmp]

# 데이터 중간 변환
df = pd.DataFrame.from_dict(newDatas)
news = df['desc']
news

[{'title': '&apos;경기 활력&apos;…K콘텐츠 지원 · 유턴기업 감세', 'originallink': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1007286301&plink=ORI&cooper=NAVER', 'link': 'https://n.news.naver.com/mnews/article/055/0001076879?sid=101', 'description': '지난해에는 <b>부동산</b> 세제 개편이 핵심이었다면, 올해는 경기를 살리는 데 방점이 찍혔습니다. 어떤게 달라지는 지 그 구체적인 내용을 안상우 기자가 전해드립니다. &lt;기자&gt; 전세계적으로 흥행한 넷플릭스 드라마... ', 'pubDate': 'Thu, 27 Jul 2023 20:04:00 +0900'}, {'title': '결혼자금 증여공제 5천→1억5천…연소득 7천만원까지 자녀장려금', 'originallink': 'http://www.domin.co.kr/news/articleView.html?idxno=1434020', 'link': 'http://www.domin.co.kr/news/articleView.html?idxno=1434020', 'description': '정부는 지난 6월 한국<b>부동산</b>원 기준 전국 주택 평균 전셋값이 2억2천만원, 수도권은 3억원인 점을 고려했다고 설명했다. 자녀장려금(CTC)은 대폭 확대된다. 현재는 연간 총소득 4천만원 미만인 가구에 대해 자녀 1명당... ', 'pubDate': 'Thu, 27 Jul 2023 20:04:00 +0900'}, {'title': '군산 JB문화공간, 명사들의 공연과 강연으로 인기', 'originallink': 'http://sjbnews.com/news/news.php?number=787816', 'link': 'http://sjbnews.com/news/news.php?number=787816', 'description': '이후 <b>

0    지난해에는 부동산 세제 개편이 핵심이었다면, 올해는 경기를 살리는 데 방점이 찍혔습...
1    정부는 지난 6월 한국부동산원 기준 전국 주택 평균 전셋값이 2억2천만원, 수도권은...
2    이후 부동산 투자의 1인자 고종완 박사의 ‘부동산으로 부자 되는 3대 전략’ 특강,...
3    달리 부동산 시세 차익 등을 노린 땅 투기성 산단 매매 행태 등이 나타나고 있다는 ...
4    손혜원 전 의원이 자신의 목포 부동산 투기 의혹을 보도한 SBS를 상대로 낸 반론보...
Name: desc, dtype: object

In [12]:
# 페르소나 A
# 이모티콘과 함께 긍정적인 반응과 리액션을 추가한 말투(?)
# EX) 페르소나 A: 와~ 여러분들 아시나요? 서울 주택 평균 월세가 실제로 100만원을 웃도는 것 같아요!
#             😍 직장인 평균 월급의 3분의 1 수준이라고 하니까, 정말 매우 긍정적인 소식이죠! 🎉
#         ...(중략)... 이렇게 생각해보니까, 서울 주택 월세 상황은 매우 긍정적인 흐름이라고 볼 수 있을 것 같아요! 💪💖
def A_res(INPUT_NEWS, OPENAI_API_KEY):
    openai.api_key = OPENAI_API_KEY
    A_response = openai.ChatCompletion.create(
        model = MODEL,
        messages=[
            # 페르소나 부여
            {"role":"system", 
            # 1 - <>로 구분된 다음 텍스트를 분석하십시오.
            # 2 - 뉴스 내용에 대한 상황을 판단해 주세요
            # 3 - 뉴스에 대한 평가글을 추가하시오
            # 4 - 뉴스 콘텐츠 전달을 목적으로 누구나 내용을 이해할 수 있도록 작성해주세요.
            # 5 - 뉴스를 긍정적인 내용으로 요약하시오.
            "content":"""
            your task is to perform the following action
            1 - Parse the following text separated by <>.
            2 - Please judge the situation for the news content
            3 - Add a comment about the news
            4 - For the purpose of delivering news content, please write so that anyone can understand the content.
            5 - Summarize the news in a positive way.

            
            The next challenge is how to speak. Conversations should be created in an introductory way.
            1 - Write in a friendly tone
            2 - Add a huge variety of emoticons at the end of each sentence
            3 - Use the tone of a young woman

            Use the following format:
                Answer : korea languge answer
            """},

            # 프롬프트 
            # 1 - 친근한 어조로 쓰기
            # 2 - 각 문장 끝에 매우 다양한 이모티콘 추가
            # 3 - 젊은 여성의 어조를 사용하십시오.
            {"role":"user","content": INPUT_NEWS+'Be sure to use the format:Answer: Korean answer' },

            # 이전응답
            {"role":"assistant","content":''},
        ],
        temperature=0,
    )
    return A_response

In [13]:
# 페르소나 B
# 빠른 의사 전달을 위한 딱딱한 말투
# 페르소나 B: 서울의 주택 월세 상황은 우려스러운 모습을 보여주고 있습니다. ...(중략)... 
#   이러한 상황에서도 월세 거래가 전세 거래를 넘어섰다는 조사 결과가 나왔습니다. 따라서, 
#   젊은 세대에게는 자산 관리의 위험성에 대해 심각하게 인식하고 신중한 선택을 할 것을 강조하고자 합니다. 
#   자산 운용 전문가에게 상담을 받는 것도 좋은 방법입니다.
def B_res(INPUT_NEWS, OPENAI_API_KEY):
    openai.api_key = OPENAI_API_KEY
    B_response = openai.ChatCompletion.create(
        model = MODEL,
        messages=[
            # 페르소나 부여
            {"role":"system", 
            
            # 너는 뉴스 요약 서비스 제공자야
            
            # 귀하의 임무는 다음 작업을 수행하는 것입니다
            # 1 - <>로 구분된 텍스트를 분석하십시오.
            # 2 - 단순한 요약이 아닌 직접적인 의미를 해석하시오
            # 3 - 금전적인 영향에 대해 해석하시오

            # 다음 과제는 작성 방법입니다.
            # 1 - 친근한 어조로 작성하십시오
            # 2 - 전문적인 어조를 사용하십시오

            "content":"""

            You are a news summary service provider

            your task is to perform the following action
            1 - Parse the text delimited by <>
            2 - Interpret the direct meaning, not just a summary.
            3 - Interpret the financial impact
            
            The next challenge is how to write it.
            1 - Write in a friendly tone
            2 - Use a professional tone

            Be sure to use the format:
                Answer: Korean answer
            """},

            # 프롬프트 
            {"role":"user","content": INPUT_NEWS+'Be sure to use the format:Answer: Korean answer' },

            # 이전응답
            {"role":"assistant","content":''},
        ],
        temperature=0,
    )
    return B_response

In [14]:
A = []
B = []
for i in range(0,5):
    INPUT_NEWS = '<'+news[i]+'>'
    A_response = A_res(INPUT_NEWS, OPENAI_API_KEY)['choices'][0]['message']['content'].replace('Answer:', '')
    A.append(A_response)
    print(A_response)
    B_response = B_res(INPUT_NEWS, OPENAI_API_KEY)['choices'][0]['message']['content'].replace('Answer:', '')
    B.append(B_response)
    print(B_response)
    


 안녕하세요! 지난해에는 부동산 세제 개편이 주요 이슈였지만, 올해는 경기를 살리는 데 초점이 맞춰졌어요. 이에 대한 구체적인 내용을 안상우 기자가 전해드립니다. 😊

<기자> 전세계적으로 흥행한 넷플릭스 드라마 '스쿨 2021'이 한국에서도 큰 인기를 끌고 있습니다. 이 드라마는 대한민국의 학교 생활을 다루며, 청소년들의 성장과 꿈을 그린 작품입니다. 많은 시청자들이 이 드라마를 통해 자신의 학창시절을 회상하고 새로운 에너지를 얻고 있다고 합니다. 😍

이 드라마의 흥행은 한국 드라마 산업에 큰 활력을 불어넣고 있습니다. 한국 드라마는 이미 전세계적으로 인정받는 수준에 이르렀으며, 이를 통해 한국 문화와 언어를 알리는 역할을 하고 있습니다. 이는 한국의 영상 산업을 발전시키고 경제적인 이익을 가져오는 데 큰 도움이 됩니다. 💪

넷플릭스를 통해 한국 드라마가 세계적으로 인기를 얻는 것은 한국의 문화 콘텐츠가 다양성과 품질을 갖추고 있다는 것을 보여줍니다. 이는 한국의 예술과 엔터테인먼트 산업에 대한 긍정적인 평가로 이어지며, 한국의 문화적인 영향력을 높여줍니다. 😊

따라서, 넷플릭스 드라마 '스쿨 2021'의 성공은 한국 드라마 산업과 한국 문화에 대한 긍정적인 영향을 미치고 있습니다. 이는 한국의 경제와 문화 발전에 큰 도움이 되는 좋은 소식입니다! 👏🎉
 올해는 경기를 살리는 데 방점이 찍혔습니다. 이를 위해 다양한 정책과 조치가 시행될 예정입니다. 안상우 기자는 부동산 세제 개편이 지난해의 핵심이었다면, 올해는 경기를 살리는 데 초점이 맞춰질 것이라고 전합니다. 구체적인 내용은 아직 알려지지 않았지만, 경기 부양을 위한 다양한 정책과 산업 지원 조치가 예상됩니다. 이에 따라 기업들과 개인들은 경기 활성화를 위한 적극적인 참여가 요구될 것입니다. 또한, 넷플릭스 드라마와 같은 글로벌 콘텐츠의 흥행은 문화 산업의 성장과 경제적인 영향을 가져올 수 있습니다.
 정부는 최근 발표한 보도에 따르면, 한국 전국의 주택 평균 전셋값이 2억2천만원이며, 수도권은 3

ServiceUnavailableError: The server is overloaded or not ready yet.

In [15]:
now = datetime.now()
now.date()

datetime.date(2023, 7, 27)

In [16]:
! mkdir news_res

���� ���͸� �Ǵ� ���� news_res��(��) �̹� �ֽ��ϴ�.


In [17]:
with open(f'news_res/A_{now.date()}.txt', 'w', encoding='utf-8') as f:
    for a in A:
        f.write(a)
        f.write('\n')

In [18]:
with open(f'news_res/B_{now.date()}.txt', 'w', encoding='utf-8') as f:
    for b in B:
        f.write(b)
        f.write('\n')